In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, \
    precision_recall_curve

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from tqdm import tqdm

import os
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

import random

/home/g/g01115/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#Dataframe  generated in analye-data.ipynb
df = pd.read_json("./Trec_data/preprocessed.json", orient='records',lines=True)

In [3]:
df

,eventID,eventType,postID,postCategories,postPriority,text,entities,extended_entities,favorite_count,hashtagEntities,...,num_unique_words,num_urls,num_words,vader compound,vader neg,vader neu,vader pos,processed,vectorized_text,regression_priority
0,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,25,0,28,0.0000,0.000,1.000,0.00,RT @CBSDenver: The copter is on the way to the...,"[-0.7409789562, 0.9875894189000001, -0.0423167...",0.00
1,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,25,0,26,0.0000,0.000,1.000,0.00,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"[-0.8112855554, 1.0994076729, 0.62657094, 0.49...",0.50
2,fireColorado2012,wildfire,217732012314861568,[FirstPartyObservation],Medium,2 wildfires in Boulder County. We can see smok...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,13,0,14,0.0000,0.000,1.000,0.00,2 wildfires in Boulder County. We can see smok...,"[-0.6315812469000001, 0.7605806589, 0.37003630...",0.50
3,fireColorado2012,wildfire,216961334129078272,[Discussion],Low,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,22,0,23,-0.6124,0.306,0.563,0.13,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"[-0.8895820379, 1.5457983017, 1.1950273514, -0...",0.25
4,fireColorado2012,wildfire,212552860590813184,[MultimediaShare],Medium,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"{'symbols': [], 'urls': [{'expanded_url': 'htt...",None,0.0,None,...,17,1,20,0.0000,0.000,1.000,0.00,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"[-0.31962126490000003, 1.1400994062, -0.346407...",0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38071,tennesseeTornadoOutbreak2020,tornado,1235236359310368768,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Hottest Google Search in 31.2 hrs. Nashville t...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,14,1,22,0.0000,0.000,1.000,0.00,Hottest Google Search in 31.2 hrs. Nashville t...,"[0.5470017195, 0.5817312002, -0.0268235132, -0...",0.25
38072,tennesseeTornadoOutbreak2020,tornado,1235337290144239616,"[ThirdPartyObservation, Location, MultimediaSh...",Low,A live report is next on the Nashville tornado...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,11,0,11,0.0000,0.000,1.000,0.00,A live report is next on the Nashville tornado...,"[1.7635256052, 0.3003855348, -0.42715665700000...",0.25
38073,tennesseeTornadoOutbreak2020,tornado,1235258820139638784,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Officials are still cleaning up after tornadoe...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,21,0,22,0.0000,0.000,1.000,0.00,Officials are still cleaning up after tornadoe...,"[0.7400860786, -0.3135090172, -0.245664984, 0....",0.25
38074,tennesseeTornadoOutbreak2020,tornado,1235253249957126144,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Putnam County: Cookeville area tornado victims...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,9,1,9,-0.3182,0.247,0.753,0.00,Putnam County: Cookeville area tornado victims...,"[0.10790167, 0.1164476871, -0.1978376657, -0.5...",0.25


# **`Generate Heldout Events`**

In [4]:
eventTypes = df['eventType'].unique()
print(eventTypes)

['wildfire' 'earthquake' 'flood' 'typhoon' 'shooting' 'bombing'
 'explosion' 'storm' 'fire' 'covid' 'tornado']


In [5]:
heldout_events = pd.DataFrame(columns=['eventID'])

#Choose heldout event and saves in the heldout_events dataframe
for event in eventTypes:
    crises = df.loc[df['eventType']==event]['eventID'].unique()
    heldout_events.loc[event]=[crises[random.choice(np.arange(crises.size))]]

heldout_events.to_json('./Trec_data/heldout_events.json')

In [6]:
#Simple read to keep the index by events
heldout_events = pd.read_json('./Trec_data/heldout_events.json')
heldout_events

,eventID
bombing,bostonBombings2013
covid,covidPhoenix2020
earthquake,indonesiaEarthquake2020
explosion,houstonExplosion2020
fire,sanFranciscoPierFire2020
flood,floodChoco2019
shooting,gilroygarlicShooting2020
storm,southeastTornadoOutbreak2020
tornado,tennesseeTornadoOutbreak2020
typhoon,hurricaneFlorence2018


# **`Model Related Methods`**

In [7]:
def train_data(data, column, heldout_ids):
    training = data.loc[~data[column].isin(heldout_ids)]
    
    return training

def test_data(data, column, heldout_ids):
    test = data.loc[data[column].isin(heldout_ids)]
    
    return test

In [8]:
def generate_model(data, features, target, modelParameters):
    
    model = RandomForestClassifier(**modelParameters)
    #model.fit(data[features].to_numpy(), data[target].to_numpy())
    model.fit(data[features], data[target])

    return model

In [9]:
def generate_model_by_events(data, features, target, modelParameters):
    modelList = pd.DataFrame(columns=['model'])
    for event in tqdm(eventTypes):
        #Create training and test dataframe
        training = train_data(df, 'eventID', heldout_crisis)
        
        #generate event specific model
        model = generate_model(training, features, target, modelParameters)
        
        #Add model to list
        modelList.loc[event] = [model]
        
        #print('')
    return modelList

In [13]:
def test_model(data, features, target, model):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-13-00bfecec9d76>, line 2)

# **`Generate Generic Variables`**

In [10]:
features = 'vectorized_text' #put in list of columns you want it to be trained on (Other features etc.)
#features.append(other_features_names)

rf_params = {
    'random_state': 1337,
    'class_weight': 'balanced',
    'n_estimators': 128, 
    'n_jobs': -1,
    'max_depth': 50,
    'max_features': 14,
    'min_samples_leaf': 33,
    'min_samples_split': 96,
    'verbose': 1
}

#Training data withholding all heldout events for general models
generalTraining = train_data(df, 'eventID', heldout_events['eventID'].tolist())
print(generalTraining.shape)

(31363, 45)


In [14]:
gen_dict = generalTraining.to_dict()

for tweet in tqdm(gen_dict['vectorized_text']):
    gen_dict['vectorized_text'][tweet] = np.array(gen_dict['vectorized_text'][tweet])
    
type(gen_dict['vectorized_text'][0])

dict

In [20]:
data = pd.DataFrame.from_dict(gen_dict)

In [30]:
for tweet in data['vectorized_text']:
    print(type(tweet))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

# **`Generate postPriority Models`**

In [36]:


def generate_model(data, features, target, modelParameters):
    
    model = RandomForestClassifier(**modelParameters)
    #model.fit(data[features].to_numpy(), data[target].to_numpy())
    
    model.fit(data[features], data[target])

    return model

#generate general model
genPrioModel = generate_model(data, features, 'postPriority', rf_params)

#generate event specific models
#specPrioModels = generate_model_by_events(df, features, 'postPriority', rf_params)

ValueError: setting an array element with a sequence.

In [ ]:
#Save postPriority models
model.save(model/postPriority.h5)

In [ ]:
for j in #whatever you want to iterate through e.g. event types:
    filename = j
    outfile = open(filename,'wb')
    #insert generate model function here
    pickle.dump(df,outfile)
    outfile.close()

# **`Generate postCategories Models`**

In [ ]:
#generate general model
catLabels = ['postCategories'] #target
genCatModel =  generate_model(generalTraining, features, catLabels, rf_params)

#generate event specific models
specCatModels = generate_model_by_events(df, features, catLabels, rf_params)

In [ ]:
#Save postCategories models

# **`Test All Models`**

In [ ]:
#Load All Models
genPrioModel
specPrioModels
genCatModel
specCatModels

In [ ]:
#Test postPriority models

In [ ]:
#Test postCategories models